In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from scipy.spatial.distance import pdist, squareform
from matplotlib import animation
import pandas as pd
plt.style.use("dark_background")

In [ ]:
"""
All the Constants in this cell.
"""

epsilon = 1
sigma = 1
m = 1  # Mass of Argon atoms
sigma12 = pow(sigma,12)
sigma6 = pow(sigma,6)
h = 0.001  # Integration step, reduced time
L = 30  # Size of lattice, reduced length
N = 100  # Number of particles
r_cutoff = 2.5 # Potential cutoff, reduced length
v_max = 5

# The velocity distribution can be a Gaussian centered around v_rms
# with the maximum velocity coming from the six-sigma rule.
# temp_Kelvin = 300
# temp_reduced = temp_Kelvin / 120
# v_rms = np.sqrt(3 * temp_reduced)  # Root Mean Squared velocity
# v_avg = np.sqrt(8 * temp_reduced / np.pi)  # average velocity
particles = np.zeros(shape=(6, N))

In [ ]:
def initialize_system(particles, seed=1327):
    np.random.seed(seed)
    x = np.repeat(np.linspace(r_cutoff, (0.5)*L - r_cutoff, int(np.sqrt(N))), int(np.sqrt(N)))
    y = np.tile(np.linspace(r_cutoff, L-r_cutoff, int(np.sqrt(N))), reps=int(np.sqrt(N)))
    particles[0:2] = np.array([x, y])
    # Random direction of velocities come from these random angles:
    theta0 = np.random.random(size=N)
    # For uniform distribution of velocities:
    velocities = np.random.uniform(low=-v_max, high=v_max, size=N)
    # For Gaussian distribution of velocities:
    # velocities = np.random.normal(loc=v_avg, scale=v_rms, size=N)
    particles[2] = velocities * np.cos(theta0)  # vx (velocity in the x direction)
    particles[3] = velocities * np.sin(theta0)  # vy (velocity in the y direction)
    # Velocity of the center of mass has to be zero:
    particles[2] = particles[2] - np.average(particles[2])
    particles[3] = particles[3] - np.average(particles[3])

def relative_distances(particles):
    # Apllies Minimum Image Convention and computes the pairwise distances between particles.
    positions = np.transpose(particles[0:2])
    # A numpy broadcasting trick to avoid using for loops.
    # (i,j,k) shows the difference between
    # the k'th coordinate of particle i and
    # the k'th coordinate of particle j.
    r_ij = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    # I don't fully understand the next line but a similar line was there in all the
    # examples I studied on the internet. It is needed for the function to work properly.
    r_ij = r_ij - np.round(r_ij / L) * L
    # Basically ternary condition for applying MIC:
    r_ij = np.where((np.abs(r_ij) > 0.5 * L), (r_ij - np.sign(r_ij) * L), r_ij)
    # The matrix of distances:
    r_ij = np.sqrt(np.sum(np.square(r_ij), axis=2))
    # Apply cut-off and sigma with two ternary conditions:
    r_ij = np.where(r_ij<r_cutoff, r_ij, np.nan)
    r_ij = np.where(r_ij>sigma, r_ij, np.nan)
    return r_ij


def acc(particles, r_ij):
    # Inverse of relative distances to the power of 6:
    ir6_ij = 1 / r_ij**6
    # Inverse of relative distances to the power of 12:
    ir12_ij = ir6_ij * ir6_ij
    acc_matrix = -24 * epsilon * (2 * sigma12 * ir12_ij - sigma6 * ir6_ij) / r_ij
    # Calculate dx and dy using a numpy trick:
    dx = particles[0, :] - particles[0, :, np.newaxis]
    dx = dx - np.round(dx / L) * L
    dy = particles[1, :] - particles[1, :, np.newaxis]
    dy = dy - np.round(dy / L) * L
    # Calculate the angle between particles:
    theta = np.arctan2(dy, dx)
    # Multiply acc_matrix with the theta to get the component of forces in the directions x and y:
    particles[4] = np.nansum(acc_matrix * np.cos(theta), axis=1)
    particles[5] = np.nansum(acc_matrix * np.sin(theta), axis=1)


def velocity_verlet(particles):
    global v_max
    # PBC Check:
    particles[0:2] = (particles[0:2]+L) % L
    # Velcity update part 1:
    particles[2:4] = particles[2:4] + 0.5 * particles[4:6] * h
    # Positions update:
    particles[0:2] = particles[0:2] + particles[2:4] * h
    # Acceleration update:
    r_ij = relative_distances(particles)
    acc(particles, r_ij)
    # Velcity update part 2:
    particles[2:4] = particles[2:4] + 0.5 * particles[4:6] * h
    # Scale velocities to avoid numerical instability:
    v_magnitude = np.linalg.norm(particles[2:4], axis=0)
    v_scale = np.minimum(v_max / v_magnitude, 1.0)
    particles[2:4] = particles[2:4] * v_scale
    # Velocity of the center of mass has to be zero:
    particles[2] = particles[2] - np.average(particles[2])
    particles[3] = particles[3] - np.average(particles[3])


def kinetic_energy(particles):
    return 0.5 * m * np.sum(particles[2:4]**2, axis=0)


def potential_energy(particles):
    r_ij = relative_distances(particles)
    # Inverse of relative distances to the power of 6:
    ir6_ij = 1 / r_ij**6
    # Inverse of relative distances to the power of 12:
    ir12_ij = ir6_ij * ir6_ij
    return 4 * epsilon * (sigma12 * ir12_ij - sigma6 * ir6_ij)


def press(particles):
    r_ij = relative_distances(particles)
    # Inverse of relative distances to the power of 6:
    ir6_ij = 1 / r_ij**6
    # Inverse of relative distances to the power of 12:
    ir12_ij = ir6_ij * ir6_ij
    # Calculate pressure using Virial's theorem:
    temp_reduced = np.mean(kinetic_energy(particles))
    PV = (N * temp_reduced) - 0.25/N * np.nansum(-24 * epsilon * (2 * sigma12 * ir12_ij - sigma6 * ir6_ij))
    return PV / L**2


def auto_correlation(xs, num_steps):
    C_j = np.zeros(num_steps)
    for j in range(1, num_steps):
        # Perason Correlation Coefficient using numpy:
        C_j[j] = np.corrcoef(xs[:-j], xs[j:])[0, 1]
    return C_j


def velocity_auto_correlation(particles, num_steps):
    # This matrix will contian the velocity of all particles through time.
    # Each column contians the evolution of the velocity of each particle:
    vs_evolution = np.zeros(shape=(num_steps, N))
    for i in range(num_steps):
        velocity_verlet(particles)
        vs_evolution[i] = particles[2]**2 + particles[3]**2
    # Each row contains the C(j) of each particle:
    C_js = np.zeros((N, num_steps))
    for i in range(N):
        C_js[i]= auto_correlation(vs_evolution[:, i].transpose(), num_steps)
    # Correlation length, all of the particles considered:
    C_j = np.mean(C_js, axis=0)
    Xi = np.argwhere(C_j<= np.exp(-1))[1][0]
    return Xi, C_j


def trajectory(particles, num_iter, num_snapshots):
    # Contains all data in 3D:
    traj = np.zeros(shape=(6, N, num_snapshots+1))
    # The first "sheet" is the initialization:
    traj[:,:,0] = particles
    for i in range(num_iter):
        velocity_verlet(particles)
        if (i+1) % (num_iter//num_snapshots) == 0:
            traj[:,:,(i+1)//(num_iter//num_snapshots)] = particles
    return traj


def take_snapshots(particles, num_iter, num_snapshots):
    ratio_left = np.zeros(num_snapshots)
    kin_energy = np.zeros(num_snapshots)
    pot_energy = np.zeros(num_snapshots)
    temperature = np.zeros(num_snapshots)
    pressure = np.zeros(num_snapshots)
    for i in range(num_iter):
        velocity_verlet(particles)
        if (i+1) % (num_iter//num_snapshots) == 0:
            kin_energy[i//(num_iter//num_snapshots)] = np.sum(kinetic_energy(particles))
            pot_energy[i//(num_iter//num_snapshots)] = np.nansum(potential_energy(particles))
            temperature[i//(num_iter//num_snapshots)] = np.mean(kinetic_energy(particles))
            pressure[i//(num_iter//num_snapshots)] = press(particles)
            ratio_left[i//(num_iter//num_snapshots)] = particles[0][particles[0] < L/2].size / N
    return ratio_left, kin_energy, pot_energy, temperature, pressure

$\Large Trajectory$

In [ ]:
# Initialize system and then run 10_000 velocity-verlet steps and store all data in the traj 3D matrix:
initialize_system(particles)
traj = trajectory(particles, num_iter=10000, num_snapshots=1000)

In [ ]:
# Write the x and y components of postitions, velocities and accelerations in seperate csv files:
pd.DataFrame(traj[0,:,:].transpose()).to_csv("trajectory_x.csv")
pd.DataFrame(traj[1,:,:].transpose()).to_csv("trajectory_y.csv")
pd.DataFrame(traj[2,:,:].transpose()).to_csv("trajectory_vx.csv")
pd.DataFrame(traj[3,:,:].transpose()).to_csv("trajectory_vy.csv")
pd.DataFrame(traj[4,:,:].transpose()).to_csv("trajectory_ax.csv")
pd.DataFrame(traj[5,:,:].transpose()).to_csv("trajectory_ay.csv")

In [ ]:
# num_iter = 1000 takes 2 seconds
initialize_system(particles, seed=1327)
num_iter = 100_000
num_snapshots = 10_000
snapshots_ratio_left,\
snapshots_kinetic_energy,\
snapshots_potential_energy,\
snapshots_temperature,\
snapshots_pressure = take_snapshots(particles, num_iter, num_snapshots)

$\Large Fraction \ of \ Particles \ in \ the \ Left \ Half \ of \ the \ Container$

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_ratio_left,
        color="m", lw=2)

ax.set_title("Fraction of Particles on the Left Half of the Container")
ax.set_xlabel(f"Reduced Time (Number of Velocity-Verlet Iterations: {num_iter})")
ax.set_ylabel("$N_{LEFT}\quad/\quadN_{TOTAL}$")
ax.set_ylim((-0.1,1.1))
ax.set_yticks(np.arange(0, 1.1, 0.1))
ax.hlines(y=0.5, xmin=0, xmax=num_iter*h, colors="orange", linestyles="--");

In [ ]:
# fig.savefig("RatioLeft.jpg")

$\Large Conservation \ of \ Energy$

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_kinetic_energy,
        color="aqua", lw=1, label="Kinetic Energy")
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_potential_energy,
        color="orange", lw=1, label="Potential Energy")
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_kinetic_energy+snapshots_potential_energy,
        color="m", lw=1, label="Total Energy")
ax.set_title("Analysing Conservation of Energy")
ax.set_xlabel(f"Reduced Time (Number of Velocity-Verlet Iterations: {num_iter})")
ax.set_ylabel("Reduced Energy")
ax.legend();

In [ ]:
# fig.savefig("EnergyConservation.jpg")

$\Large Auto-Correlation$

In [ ]:
# This cell takes 1.5 minutes to execute.
initialize_system(particles)
num_steps = 10_000
Xi, C_j = velocity_auto_correlation(particles, num_steps=num_steps)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(np.linspace(0, 10_000*h, num_steps-1), C_j[1:],
        color="aqua", lw=3, label="Auto-Correlation")
ax.set_title(f"Auto-Correlation of Velocities; $\\xi = 1.095$")
ax.set_xlabel(f"Reduced Time (Number of Velocity-Verlet Iterations: {num_steps})")
ax.set_ylabel("$C(j)$")
ax.hlines(y=np.exp(-1), xmin=0, xmax=10_000*h, colors="orange", linestyles="--", label="$e^{-1}$")
ax.vlines(x=1.095, ymin=0, ymax=1, linestyles="--", label="$\\xi$")
ax.set_xticks([0, 2, 4, 6, 8, 10, 1.095])
ax.legend();

In [ ]:
# fig.savefig("Autocorrelation.jpg")

$\Large Temperature \ vs. \ Time$

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_temperature,
        color="aqua", lw=1)
temperature = np.nanmean(snapshots_temperature[2000:])
std = np.std(snapshots_temperature[2000:])
ax.set_title(f"Temperature vs. Time\n$T*={temperature:.3f} \pm {std:.3f}$")
ax.set_xlabel(f"Reduced Time (Number of Velocity-Verlet Iterations: {num_iter})")
ax.set_ylabel("Reduced Termperature");

In [ ]:
# fig.savefig("Temperature_vs_Time.jpg")

$\Large Pressure \ vs. \ Time$

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(np.linspace(0, num_iter*h, num_snapshots), snapshots_pressure,
        color="aqua", lw=1)
pressure = np.nanmean(snapshots_pressure[2000:])
std = np.std(snapshots_pressure[2000:])
ax.set_title(f"Pressure vs. Time\n$P*={pressure:.4f} \pm {std:.4f}$")
ax.set_xlabel(f"Reduced Time (Number of Velocity-Verlet Iterations: {num_iter})")
ax.set_ylabel("Reduced Pressure");

In [ ]:
# fig.savefig("Pressure_vs_Time.jpg")

$\Large Van \ der \ Waals \ Constants$

In [ ]:
# Takes 6 minutes
# Reduced values for v_max:
v_maxs = np.arange(5, 1, -0.1)
# v_maxs = [5, 4, 3, 1]
# A matrix to contain temperature in the first row and pressure in the second row for each value of v_max:
T_and_P = np.zeros(shape=(2, len(v_maxs)))
v_max = v_maxs[0]
initialize_system(particles)
for i in range(len(v_maxs)):
    v_max = v_maxs[i]
    P = 0
    T = 0
    if i==0:
        for j in range(2000):
            velocity_verlet(particles)
    for j in range(3000):
        velocity_verlet(particles)
        P += press(particles)
        T += np.mean(kinetic_energy(particles))
    T_and_P[0, i] = T / 3000
    T_and_P[1, i] = P / 3000


In [ ]:
coefs = np.polynomial.polynomial.polyfit(T_and_P[0], T_and_P[1], deg=1)
ffit = np.polynomial.polynomial.polyval(T_and_P[0], coefs)

fig, ax = plt.subplots()


ax.scatter(T_and_P[0], T_and_P[1], color="orange", label="Data", zorder=1)
ax.plot(T_and_P[0], ffit, "m--", label=f"Fit: y = {coefs[1]:.6f}x + ({coefs[0]:.6f})", zorder=2)
ax.set_title("Pressure vs. Temperature")
ax.set_xlabel("Reduced Temperature", fontsize=12)
ax.set_ylabel("Reduced Pressure", fontsize=12)
ax.legend();

In [ ]:
# fig.savefig("Pressure_vs_Temperature.jpg")

In [ ]:
"""
Turning to SI units:
"""
k_B = 1.38 * 10**-23
N_A = 6.022 * 10**23
R = N_A * k_B
epsilon_SI = 120 * k_B  # From Wikipedia
sigma_SI = 0.34 * 10**-9  # From Wikipedia
V_molar = 22.4 * 10**-3  # From Wikipedia
Ts_SI = T_and_P[0] * epsilon_SI / k_B  # Temperatures in SI
Ps_SI = T_and_P[1] * epsilon_SI / sigma_SI**3  # Pressures in SI
coefs_SI = np.polynomial.polynomial.polyfit(Ts_SI, Ps_SI, deg=1)

a = V_molar**2 * -coefs_SI[0]
b = V_molar - (R / coefs_SI[1])
a_wikipedia = 1.355
b_wikipedia = 0.03201

print(f"a = {a:.3f}    ERORR = {np.abs((a-a_wikipedia)/a_wikipedia*100):.3f}%")
print(f"b = {b:.3f}    ERORR = {np.abs((b-b_wikipedia)/b_wikipedia*100):.3f}%")

$\Large ANIMATION$

$\Large Gas$

In [ ]:
# Takes 1 minute.
plt.rcParams["animation.html"] = "jshtml"

v_max = 5
h = 0.001
# Initiating the plot:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim((-1, L+1))
ax.set_ylim((-1, L+1))


# Create an empty point object with no data.
# It will be updated during each frame of the animation.
(point,) = ax.plot([], [], "o", color="aqua", ms=5)

# initialization function: plot the background of each frame.
# There was a similar function in all the example codes I studied; must be the standard way!
def init():
    point.set_data([], [])
    return point,


# The animation function. This is called sequentially.
def animate(i):
    # Adjusting for the first frame:
    if i==0:
        initialize_system(particles, seed=13270)
        ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {0}")
        point.set_data(particles[0], particles[1])
        return point,
    i = i-1
    # Take 100 Velocity-Verlet steps and then set a frame:
    for _ in range(100):
        velocity_verlet(particles)
    ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {100*(i+1)*h:.1f}")
    point.set_data(particles[0], particles[1])  # The standard way to update x and y.
    return point,


# Finally moving on to the animator function.
# frames parameter determines the number of particles that are depositted in each frame.
# If we add one single particle in each frame, a lot of time and a lot of memory will be used.
# blit=True means only re-draw the parts that have changed; this helps increase the speed of the program.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=300, blit=True)
plt.close("all")
anim

In [ ]:
# Saving the animation in mp4:
# Takes 1 minute.
dpi = 300
writer = animation.writers["ffmpeg"](fps=30)
anim.save("GasPhase_vmax5.mp4",writer=writer,dpi=dpi)

$\Large Liquid$

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

v_max = 2
h = 0.001
# Initiating the plot:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim((-1, L+1))
ax.set_ylim((-1, L+1))


# Create an empty point object with no data.
# It will be updated during each frame of the animation.
(point,) = ax.plot([], [], "o", color="aqua", ms=5)

# initialization function: plot the background of each frame.
# There was a similar function in all the example codes I studied; must be the standard way!
def init():
    point.set_data([], [])
    return point,


# The animation function. This is called sequentially.
def animate(i):
    # Adjusting for the first frame:
    if i==0:
        initialize_system(particles, seed=13270)
        ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {0}")
        point.set_data(particles[0], particles[1])
        return point,
    i = i-1
    # Take 100 Velocity-Verlet steps and then set a frame:
    for _ in range(100):
        velocity_verlet(particles)
    ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {100*(i+1)*h:.1f}")
    point.set_data(particles[0], particles[1])  # The standard way to update x and y.
    return point,


# Finally moving on to the animator function.
# frames parameter determines the number of particles that are depositted in each frame.
# If we add one single particle in each frame, a lot of time and a lot of memory will be used.
# blit=True means only re-draw the parts that have changed; this helps increase the speed of the program.
anim_liquid = animation.FuncAnimation(fig, animate, init_func=init, frames=300, blit=True)
plt.close("all")
anim_liquid

In [ ]:
# Saving the animation in mp4:
# Takes 1 minute.
dpi = 300
writer = animation.writers["ffmpeg"](fps=30)
anim_liquid.save("LquidPhase_vmax2.mp4",writer=writer,dpi=dpi)

$\Large Solid$

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

v_max = 0.1
h = 0.001
# Initiating the plot:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim((-1, L+1))
ax.set_ylim((-1, L+1))


# Create an empty point object with no data.
# It will be updated during each frame of the animation.
(point,) = ax.plot([], [], "o", color="aqua", ms=5)

# initialization function: plot the background of each frame.
# There was a similar function in all the example codes I studied; must be the standard way!
def init():
    point.set_data([], [])
    return point,


# The animation function. This is called sequentially.
def animate(i):
    # Adjusting for the first frame:
    if i==0:
        initialize_system(particles, seed=13270)
        ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {0}")
        point.set_data(particles[0], particles[1])
        return point,
    i = i-1
    # Take 100 Velocity-Verlet steps and then set a frame:
    for _ in range(100):
        velocity_verlet(particles)
    ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {100*(i+1)*h:.1f}")
    point.set_data(particles[0], particles[1])  # The standard way to update x and y.
    return point,


# Finally moving on to the animator function.
# frames parameter determines the number of particles that are depositted in each frame.
# If we add one single particle in each frame, a lot of time and a lot of memory will be used.
# blit=True means only re-draw the parts that have changed; this helps increase the speed of the program.
anim_solid = animation.FuncAnimation(fig, animate, init_func=init, frames=300, blit=True)
plt.close("all")
anim_solid

In [ ]:
# Saving the animation in mp4:
# Takes 1 minute.
dpi = 300
writer = animation.writers["ffmpeg"](fps=30)
anim_solid.save("SolidPhase_vmax01.mp4",writer=writer,dpi=dpi)

$\Large Phase \ Transition$

In [ ]:
v_max = 4
initialize_system(particles, seed=13270)
for _ in range(15000):
    velocity_verlet(particles)

In [ ]:
# v_maxs = [3, 2.5, 2, 1.5, 1.1, 1, 0.5, 0.2, 0.1]
        #   1.3, 1.2, 1.1, 1. , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.09, 0.05]
v_maxs = np.arange(2, 0, -0.05)
# v_maxs = [2, 1, 0.1]
plt.rcParams["animation.html"] = "jshtml"

v_max = 4
h = 0.001
# Initiating the plot:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim((-1, L+1))
ax.set_ylim((-1, L+1))


# Create an empty point object with no data.
# It will be updated during each frame of the animation.
(point,) = ax.plot([], [], "o", color="aqua", ms=5)

# initialization function: plot the background of each frame.
# There was a similar function in all the example codes I studied; must be the standard way!
def init():
    point.set_data([], [])
    return point,


# The animation function. This is called sequentially.
def animate(i):
    # elif i%10==0:
        # global v_max
        # v_max = v_maxs[i//10]
    global v_max
    v_max = v_maxs[i//10]
    # Take 100 Velocity-Verlet steps and then set a frame:
    for _ in range(200):
        velocity_verlet(particles)
        v_max
    ax.set_title(f"Temperature = {np.mean(kinetic_energy(particles))*120:.2f} K\nt* = {100*(i)*h:.1f}")
    point.set_data(particles[0], particles[1])  # The standard way to update x and y.
    return point,


# Finally moving on to the animator function.
# frames parameter determines the number of particles that are depositted in each frame.
# If we add one single particle in each frame, a lot of time and a lot of memory will be used.
# blit=True means only re-draw the parts that have changed; this helps increase the speed of the program.
anim_phase_transition = animation.FuncAnimation(fig, animate, init_func=init, frames=400, blit=True)
plt.close("all")
anim_phase_transition

In [ ]:
# Saving the animation in mp4:
# Takes 1 minute.
dpi = 300
writer = animation.writers["ffmpeg"](fps=40)
anim_phase_transition.save("PhaseTransitions.mp4",writer=writer,dpi=dpi)